In [8]:
import os
import sys
import SimpleITK as sitk
import numpy as np
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import medpy.metric as medmetric
from PIL import Image
import cv2
from skimage import morphology
from matplotlib import pyplot as plot
import pandas as pd
import xlrd

## 计算狭窄率
- 对mask进行腐蚀操作，去掉外膜得到正常血管壁的厚度


In [9]:
model_name = "label"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

pn_all = {}
pne_all = {}
mask_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == file_name[:-7])][0][0][1])

    pn = []
    pne = []
    mask = {}
    mask_out_list = []
    mask_in_list = []
    mask_both_list = []
    mask_both_erode_list = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=spacing // 10)  # 外膜腐蚀mask

        mask_out_list.append(out_mask)
        mask_in_list.append(in_mask)
        mask_both_list.append(both_mask)
        mask_both_erode_list.append(both_mask_erode)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}'
            )

    mask["out"] = mask_out_list
    mask["in"] = mask_in_list
    mask["both"] = mask_both_list
    mask["both_erode"] = mask_both_erode_list

    mask_all[file_name] = mask
    pn_all[file_name] = pd.Series(pn)
    pne_all[file_name] = pd.Series(pne)

if True:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_all)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_all)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [10]:
model_name = "3dunet"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

pn_3dunet_all = {}
pne_3dunet_all = {}
mask_3dunet_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == file_name[:-7])][0][0][1])

    pn = []
    pne = []
    mask = {}
    mask_out_list = []
    mask_in_list = []
    mask_both_list = []
    mask_both_erode_list = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=spacing // 10)  # 外膜腐蚀mask

        mask_out_list.append(out_mask)
        mask_in_list.append(in_mask)
        mask_both_list.append(both_mask)
        mask_both_erode_list.append(both_mask_erode)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}'
            )

    mask["out"] = mask_out_list
    mask["in"] = mask_in_list
    mask["both"] = mask_both_list
    mask["both_erode"] = mask_both_erode_list

    mask_3dunet_all[file_name] = mask
    pn_3dunet_all[file_name] = pd.Series(pn)
    pne_3dunet_all[file_name] = pd.Series(pne)

if True:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_3dunet_all)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_3dunet_all)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [11]:
model_name = "rvm"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

pn_rvm_all = {}
pne_rvm_all = {}
mask_rvm_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == file_name[:-7])][0][0][1])

    pn = []
    pne = []
    mask = {}
    mask_out_list = []
    mask_in_list = []
    mask_both_list = []
    mask_both_erode_list = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=spacing // 10)  # 外膜腐蚀mask

        mask_out_list.append(out_mask)
        mask_in_list.append(in_mask)
        mask_both_list.append(both_mask)
        mask_both_erode_list.append(both_mask_erode)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}'
            )

    mask["out"] = mask_out_list
    mask["in"] = mask_in_list
    mask["both"] = mask_both_list
    mask["both_erode"] = mask_both_erode_list

    mask_rvm_all[file_name] = mask
    pn_rvm_all[file_name] = pd.Series(pn)
    pne_rvm_all[file_name] = pd.Series(pne)

if True:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_rvm_all)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_rvm_all)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [12]:
model_name = "transbts"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

pn_transbts_all = {}
pne_transbts_all = {}
mask_transbts_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == file_name[:-7])][0][0][1])

    pn = []
    pne = []
    mask = {}
    mask_out_list = []
    mask_in_list = []
    mask_both_list = []
    mask_both_erode_list = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=spacing // 10)  # 外膜腐蚀mask

        mask_out_list.append(out_mask)
        mask_in_list.append(in_mask)
        mask_both_list.append(both_mask)
        mask_both_erode_list.append(both_mask_erode)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}'
            )

    mask["out"] = mask_out_list
    mask["in"] = mask_in_list
    mask["both"] = mask_both_list
    mask["both_erode"] = mask_both_erode_list

    mask_transbts_all[file_name] = mask
    pn_transbts_all[file_name] = pd.Series(pn)
    pne_transbts_all[file_name] = pd.Series(pne)

if True:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_transbts_all)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_transbts_all)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

## 计算壁厚
- 用1.5mm阈值来区分当前帧图像有无斑块，将有斑块帧的狭窄率绘制成图


In [13]:
def remove_isolate(inputs, threshold_area=0.5):
    mask = np.zeros((inputs.shape[0], inputs.shape[1]), dtype=np.uint8)
    mask[np.sum(inputs, axis=-1) > 0] = 1
    mask = morphology.remove_small_objects(mask.astype(np.bool8),
                                           np.sum(mask) * threshold_area,
                                           connectivity=8).astype(np.uint8)
    # outputs = np.expand_dims(mask, -1) * inputs
    outputs = mask * inputs

    return outputs


def minimum_external_circle(img):
    contours, _ = cv2.findContours(img, cv2.RETR_TREE,
                                   cv2.CHAIN_APPROX_NONE)  # 提取轮廓
    img = np.expand_dims(img, -1).repeat(3, -1)

    cnt = contours[0]
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))  # 最小内接圆圆心
    radius = int(radius)  # 最小内接圆直径
    cv2.circle(img, center, radius, (0, 255, 0), 2)
    cv2.circle(img, center, 1, (0, 255, 0), 2)
    return (x, y), radius, img


# 把笛卡尔坐标转化为极坐标
def to_polar(vector, x, y):
    v_length = np.sqrt((vector[1] - int(x))**2 + (vector[0] - int(y))**2)
    v_angle = np.arctan2(vector[0] - int(y), vector[1] - int(x))
    return (v_length, np.around(v_angle, 2))


def circle_max_distance(polar):
    angle_range = np.linspace(-3.14, 3.14, int((3.14 * 2) / 0.01 + 1))
    polar = np.array(polar)
    polar_angle = polar[:, 1]
    radial = []
    for angle in angle_range:
        idx_list = np.argwhere(polar_angle == np.around(angle, 2))
        if len(idx_list) != 0:
            distance = np.max(polar[idx_list][:, 0, 0]) - np.min(
                polar[idx_list][:, 0, 0])
            radial.append(int(distance))

    return max(radial)

In [14]:
data_distance = {}
for i, data_tuple in enumerate(list(mask_all.items())):
    data = data_tuple[1]["out"]
    name = data_tuple[0]
    frame_distance = []
    for j, frame in enumerate(data):
        frame_pad = np.pad(frame, ((100, 100), (100, 100)), 'constant', constant_values=0)
        frame_pad = cv2.morphologyEx(frame_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8))
        frame = frame_pad[100:-100, 100:-100]
        frame = remove_isolate(frame)

        area_pad = np.pad(data_tuple[1]["both"][j],
                          ((100, 100), (100, 100)),
                          'constant',
                          constant_values=0)
        area_pad = cv2.morphologyEx(area_pad,
                                    cv2.MORPH_CLOSE,
                                    kernel=np.ones((15, 15), np.uint8))
        area = area_pad[100:-100, 100:-100]
        area = remove_isolate(area)


        (x, y), radius, visual_frame = minimum_external_circle(np.array(area, dtype=np.uint8))

        frame_index = np.argwhere(frame == 1)

        polar = []
        for idx in range(len(frame_index)):
            polar.append(to_polar(frame_index[idx], x, y))
        polar = sorted(polar, key=lambda item: item[1])

        frame_distance.append(circle_max_distance(polar))

        # plot.figure(j)
        # plot.imshow(visual_frame)
        # plot.title(str(circle_max_distance(polar)))

    data_distance[name] = frame_distance

In [15]:
data_rvm_distance = {}
for i, data_tuple in enumerate(list(mask_rvm_all.items())):
    data = data_tuple[1]["out"]
    name = data_tuple[0]
    frame_distance = []
    for j, frame in enumerate(data):
        frame_pad = np.pad(frame, ((100, 100), (100, 100)), 'constant', constant_values=0)
        frame_pad = cv2.morphologyEx(frame_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8))
        frame = frame_pad[100:-100, 100:-100]
        frame = remove_isolate(frame)

        area_pad = np.pad(data_tuple[1]["both"][j],
                          ((100, 100), (100, 100)),
                          'constant',
                          constant_values=0)
        area_pad = cv2.morphologyEx(area_pad,
                                    cv2.MORPH_CLOSE,
                                    kernel=np.ones((15, 15), np.uint8))
        area = area_pad[100:-100, 100:-100]
        area = remove_isolate(area)


        (x, y), radius, visual_frame = minimum_external_circle(np.array(area, dtype=np.uint8))

        frame_index = np.argwhere(frame == 1)

        polar = []
        for idx in range(len(frame_index)):
            polar.append(to_polar(frame_index[idx], x, y))
        polar = sorted(polar, key=lambda item: item[1])

        try:
            frame_distance.append(circle_max_distance(polar))
        except:
            frame_distance.append(0)
            print(name)

        # plot.figure(j)
        # plot.imshow(visual_frame)
        # plot.title(str(circle_max_distance(polar)))

    data_rvm_distance[name] = frame_distance

In [41]:
data_3dunet_distance = {}
for i, data_tuple in enumerate(list(mask_3dunet_all.items())):
    data = data_tuple[1]["out"]
    name = data_tuple[0]
    frame_distance = []
    for j, frame in enumerate(data):
        frame_pad = np.pad(frame, ((100, 100), (100, 100)), 'constant', constant_values=0)
        frame_pad = cv2.morphologyEx(frame_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8))
        frame = frame_pad[100:-100, 100:-100]
        frame = remove_isolate(frame)

        area_pad = np.pad(data_tuple[1]["both"][j],
                          ((100, 100), (100, 100)),
                          'constant',
                          constant_values=0)
        area_pad = cv2.morphologyEx(area_pad,
                                    cv2.MORPH_CLOSE,
                                    kernel=np.ones((15, 15), np.uint8))
        area = area_pad[100:-100, 100:-100]
        area = remove_isolate(area)


        (x, y), radius, visual_frame = minimum_external_circle(np.array(area, dtype=np.uint8))

        frame_index = np.argwhere(frame == 1)

        polar = []
        for idx in range(len(frame_index)):
            polar.append(to_polar(frame_index[idx], x, y))
        polar = sorted(polar, key=lambda item: item[1])

        try:
            frame_distance.append(circle_max_distance(polar))
        except:
            frame_distance.append(0)
            print(name)

        # plot.figure(j)
        # plot.imshow(visual_frame)
        # plot.title(str(circle_max_distance(polar)))

    data_3dunet_distance[name] = frame_distance

202203140858390011VAS.nii.gz


In [42]:
print(len(data_3dunet_distance["202203140858390011VAS.nii.gz"]))
print(len(list(mask_3dunet_all["202203140858390011VAS.nii.gz"]['out'])))

41
41


In [36]:
i = 0
for image in list(mask_3dunet_all["202203140858390011VAS.nii.gz"]['out']):
    plot.imsave(f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/未命名文件夹/{i}.png", image)
    i += 1

In [45]:
data_transbts_distance = {}
for i, data_tuple in enumerate(list(mask_transbts_all.items())):
    data = data_tuple[1]["out"]
    name = data_tuple[0]
    frame_distance = []
    for j, frame in enumerate(data):
        frame_pad = np.pad(frame, ((100, 100), (100, 100)), 'constant', constant_values=0)
        frame_pad = cv2.morphologyEx(frame_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8))
        frame = frame_pad[100:-100, 100:-100]
        frame = remove_isolate(frame)

        area_pad = np.pad(data_tuple[1]["both"][j],
                          ((100, 100), (100, 100)),
                          'constant',
                          constant_values=0)
        area_pad = cv2.morphologyEx(area_pad,
                                    cv2.MORPH_CLOSE,
                                    kernel=np.ones((15, 15), np.uint8))
        area = area_pad[100:-100, 100:-100]
        area = remove_isolate(area)


        (x, y), radius, visual_frame = minimum_external_circle(np.array(area, dtype=np.uint8))

        frame_index = np.argwhere(frame == 1)

        polar = []
        for idx in range(len(frame_index)):
            polar.append(to_polar(frame_index[idx], x, y))
        polar = sorted(polar, key=lambda item: item[1])

        try:
            frame_distance.append(circle_max_distance(polar))
        except:
            frame_distance.append(0)
            print(name)

        # plot.figure(j)
        # plot.imshow(visual_frame)
        # plot.title(str(circle_max_distance(polar)))

    data_transbts_distance[name] = frame_distance

202203081456170036VAS.nii.gz
202204020917510011VAS.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz
IM_0010.nii.gz


In [44]:
print(len(data_transbts_distance['202203081456170036VAS.nii.gz']))
print(len(list(mask_transbts_all["202203081456170036VAS.nii.gz"]['out'])))

70
70


In [32]:
i = 0
for image in list(mask_transbts_all["202203081456170036VAS.nii.gz"]['out']):
    plot.imsave(
        f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/未命名文件夹/{i}.png",
        image)
    i += 1

In [57]:
# 卡阈值选出斑块帧
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

for k, v in data_distance.items():
    voxel = int(spacing_pixels[np.argwhere(spacing_pixels[:, 0] == k[:-7])][0][0][1]) / 5
    for idx, data in enumerate(v):
        if data / voxel <= 1.5:
            pne_all[k][idx] = 0
        else:
            pass


In [58]:
# 卡阈值选出斑块帧
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

for k, v in data_rvm_distance.items():
    voxel = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == k[:-7])][0][0][1]) / 5
    for idx, data in enumerate(v):
        if data / voxel <= 1.5:
            pne_rvm_all[k][idx] = 0
        else:
            pass


In [59]:
# 卡阈值选出斑块帧
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

for k, v in data_3dunet_distance.items():
    voxel = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == k[:-7])][0][0][1]) / 5
    for idx, data in enumerate(v):
        if data / voxel <= 1.5:
            pne_3dunet_all[k][idx] = 0
        else:
            pass


In [60]:
# 卡阈值选出斑块帧
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

for k, v in data_transbts_distance.items():
    voxel = int(spacing_pixels[np.argwhere(
        spacing_pixels[:, 0] == k[:-7])][0][0][1]) / 5
    for idx, data in enumerate(v):
        if data / voxel <= 1.5:
            pn_transbts_all[k][idx] = 0
        else:
            pass


In [ ]:
save_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/figs_1.5mm"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for idx in range(len(pne_all)):
    assert sorted(list(pne_all.keys()))[idx] == sorted(
            list(pn_transbts_all.keys()))[idx] == sorted(list(
            pne_3dunet_all.keys()))[idx] == sorted(list(pne_rvm_all.keys()))[idx]

    title = sorted(list(pne_all.keys()))[idx][:-7]

    fig_x = list(pne_all[sorted(list(pne_all.keys()))[idx]].index)
    fig_y = list(pne_all[sorted(list(pne_all.keys()))[idx]].values)

    fig_x_1 = list(pne_3dunet_all[sorted(list(
        pne_3dunet_all.keys()))[idx]].index)
    fig_y_1 = list(pne_3dunet_all[sorted(list(
        pne_3dunet_all.keys()))[idx]].values)

    fig_x_2 = list(pn_transbts_all[sorted(list(
        pn_transbts_all.keys()))[idx]].index)
    fig_y_2 = list(pn_transbts_all[sorted(list(
        pn_transbts_all.keys()))[idx]].values)

    fig_x_3 = list(pne_rvm_all[sorted(list(pne_rvm_all.keys()))[idx]].index)
    fig_y_3 = list(pne_rvm_all[sorted(list(pne_rvm_all.keys()))[idx]].values)

    plot.rcParams.update({"font.size": 10})
    fig = plot.figure(idx, figsize=(18, 6), dpi=300)
    ax = fig.add_subplot(111)

    ax.set(title=title,
           xlabel='frame',
           ylabel='diameter stenosis',
           ylim=[0, 1])

    ax.plot(fig_x, fig_y, 'm', label='label')

    ax.plot(fig_x_1, fig_y_1, 'g', label='3dunet')

    ax.plot(fig_x_2, fig_y_2, 'c', label='transbts')

    ax.plot(fig_x_3, fig_y_3, 'b', label='rvm')

    ax.legend()
    ax.grid(True)

    fig.savefig(f'{save_dir}/{title}.png')

## 计算指标

In [78]:
def _take_channels(*xs, ignore_channels=None):
    if ignore_channels is None:
        return xs
    else:
        channels = [channel for channel in range(xs[0].shape[1]) if channel not in ignore_channels]
        xs = [torch.index_select(x, dim=1, index=torch.tensor(channels).to(x.device)) for x in xs]
        return xs


def _threshold(x, threshold=None):
    if threshold is not None:
        return (x > threshold).type(x.dtype)
    else:
        return x


class AccuracyProportion(nn.Module):
    def __init__(self, proportion=0.5, threshold=None, ignore_channels=None, eps=1e-7):
        super().__init__()
        self.proportion = proportion
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.eps = eps

    def forward(self, pr, gt):
        assert pr.shape[0] == gt.shape[0]

        pr = _threshold(pr, threshold=self.threshold)
        pr, gt = _take_channels(pr, gt, ignore_channels=self.ignore_channels)

        sum_nums = pr.shape[0]
        nums = 0
        for idx in range(sum_nums):
            intersection = torch.sum(gt[idx] * pr[idx])
            union = torch.sum(gt[idx]) + torch.sum(pr[idx]) - intersection + self.eps
            score = (intersection + self.eps) / union

            if score > self.proportion:
                nums += 1
        ap_score = nums / sum_nums

        return ap_score


class ExpectedAverageOverlap(nn.Module):
    def __init__(self, threshold=None, ignore_channels=None, time_dim=0, eps=1e-7):
        super().__init__()
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.time_dim = time_dim
        self.eps = eps

    def forward(self, pr, gt):
        assert pr.shape[self.time_dim] == gt.shape[self.time_dim]

        pr = _threshold(pr, threshold=self.threshold)
        pr, gt = _take_channels(pr, gt, ignore_channels=self.ignore_channels)

        nums = pr.shape[self.time_dim]
        score = 0
        for idx in range(nums):
            temp = 0
            for k in range(idx):
                tp = torch.sum((gt[k] == pr[k]).type(pr.dtype))
                temp += tp / (gt[k].view(-1).shape[0] + self.eps)

            score += temp / (idx+1)
        eao_score = score / nums

        return eao_score
    

class DistanceMetric(nn.Module):
        def __init__(self, method_name="hd", voxelspacing=None, threshold=None, ignore_channels=None):
            super().__init__()
            self.method_name = method_name
            self.voxelspacing = voxelspacing
            self.threshold = threshold
            self.ignore_channels = ignore_channels
    
        def forward(self, pr, gt):
            assert len(pr.shape) == len(gt.shape) == 2, "输入数据应该是一个背景纯黑的面"

            pr = _threshold(pr, threshold=self.threshold)
            pr, gt = _take_channels(pr, gt, ignore_channels=self.ignore_channels)

            pr = pr.cpu().numpy()
            gt = gt.cpu().numpy()

            if self.method_name == "asd":
                distance_score = medmetric.binary.asd(pr, gt, voxelspacing=self.voxelspacing)
            elif self.method_name == "assd":
                distance_score = medmetric.binary.assd(pr, gt, voxelspacing=self.voxelspacing)
            elif self.method_name == "hd":
                distance_score = medmetric.binary.hd(pr, gt, voxelspacing=self.voxelspacing)
            elif self.method_name == "hd95":    
                distance_score = medmetric.binary.hd95(pr, gt, voxelspacing=self.voxelspacing)
            else:
                print(f"方法{self.method_name}不存在，请设置方法为asd、assd、hd或hd95")
                sys.exit()
            
            return distance_score


def metric_functions(loss_name):
    if loss_name == 'dice':
        return smp.utils.metrics.Fscore(beta=1, threshold=0.5)
    elif loss_name == 'ac':
        return smp.utils.metrics.Accuracy(threshold=0.5)
    elif loss_name == 'pre':
        return smp.utils.metrics.Precision(threshold=0.5)
    elif loss_name == 'r':
        return smp.utils.metrics.Recall(threshold=0.5)
    elif loss_name == 'f1':
        return smp.utils.metrics.Fscore(beta=1, threshold=0.5)
    elif loss_name == 'iou':
        return smp.utils.metrics.IoU(threshold=0.5)
    elif loss_name == 'ap@0.5':
        return AccuracyProportion(proportion=0.5, threshold=0.5)
    elif loss_name == 'ap@0.7':
        return AccuracyProportion(proportion=0.7, threshold=0.5)
    elif loss_name == 'eao':
        return ExpectedAverageOverlap(threshold=0.5)
    elif loss_name == 'asd':
        return DistanceMetric(method_name="asd", threshold=0.5)
    elif loss_name == 'assd':
        return DistanceMetric(method_name="assd", threshold=0.5)
    elif loss_name == 'hd':
        return DistanceMetric(method_name="hd", threshold=0.5)
    elif loss_name == 'hd95':
        return DistanceMetric(method_name="hd95", threshold=0.5)
    else:
        print('The loss function name: {} is invalid'.format(loss_name))
        sys.exit()


In [77]:
print(list(mask_transbts_all.items())[0][1].keys())
print(list(mask_transbts_all.items())[0][0])
a = torch.tensor(list(mask_transbts_all.items())[0][1]['both'])

dict_keys(['out', 'in', 'both', 'both_erode'])
02220820210305_XJ_20210305082931543.nii.gz


/var/folders/jb/g3ylc11574sd3g6w3t57g5rm0000gn/T/ipykernel_18799/2628242762.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1662188943573/work/torch/csrc/utils/tensor_new.cpp:233.)
  a = torch.tensor(list(mask_transbts_all.items())[0][1]['both'])


In [119]:
def count_index(mask_data, mask_label, spacing_pixels, model_name, save_dir):

    dice_in_list = []
    dice_bothunique_list = []

    ac_in_list = []
    ac_bothunique_list = []

    pre_in_list = []
    pre_bothunique_list = []

    r_in_list = []
    r_bothunique_list = []

    f1_in_list = []
    f1_bothunique_list = []

    iou_in_list = []
    iou_bothunique_list = []

    ap_05_in_list = []
    ap_05_bothunique_list = []

    ap_07_in_list = []
    ap_07_bothunique_list = []

    eao_in_list = []
    eao_bothunique_list = []

    asd_in_list = []
    asd_bothunique_list = []

    assd_in_list = []
    assd_bothunique_list = []

    hd_in_list = []
    hd_bothunique_list = []

    hd95_in_list = []
    hd95_bothunique_list = []


    for i, file_name in enumerate(list(mask_data.keys())):

        data_in = mask_data[file_name]["in"]
        label_in = mask_label[file_name]["in"]
        data_both = mask_data[file_name]["both"]
        label_both = mask_label[file_name]["both"]

        data_in = torch.tensor(data_in)
        label_in = torch.tensor(label_in)
        data_both = torch.tensor(data_both)
        label_both = torch.tensor(label_both)

        if False:
            label_in = cv2.resize(np.array(label_in).transpose((1, 2, 0)), (128, 128),
                       interpolation=cv2.INTER_NEAREST).transpose((2, 0, 1))
            label_in = torch.tensor(label_in).contiguous()
            label_both = cv2.resize(np.array(label_both).transpose((1, 2, 0)), (128, 128),
                       interpolation=cv2.INTER_NEAREST).transpose((2, 0, 1))
            label_both = torch.tensor(label_both).contiguous()

        voxel = 5 / int(spacing_pixels[np.argwhere(
            spacing_pixels[:, 0] == file_name[:-7])][0][0][1])

        if True:
            # 计算指标
            dice_in_all = metric_functions("dice")(data_in, label_in)
            dice_in_list.append(dice_in_all.cpu())
            dice_bothunique = metric_functions("dice")(data_both, label_both)
            dice_bothunique_list.append(dice_bothunique.cpu())

            ac_in = metric_functions("ac")(data_in, label_in)
            ac_in_list.append(ac_in.cpu())
            ac_bothunique = metric_functions("ac")(data_both, label_both)
            ac_bothunique_list.append(ac_bothunique.cpu())

            pre_in = metric_functions("pre")(data_in, label_in)
            pre_in_list.append(pre_in.cpu())
            pre_bothunique = metric_functions("pre")(data_both, label_both)
            pre_bothunique_list.append(pre_bothunique.cpu())

            r_in = metric_functions("r")(data_in, label_in)
            r_in_list.append(r_in.cpu())
            r_bothunique = metric_functions("r")(data_both, label_both)
            r_bothunique_list.append(r_bothunique.cpu())

            f1_in = metric_functions("f1")(data_in, label_in)
            f1_in_list.append(f1_in.cpu())
            f1_bothunique = metric_functions("f1")(data_both, label_both)
            f1_bothunique_list.append(f1_bothunique.cpu())

            iou_in = metric_functions("iou")(data_in, label_in)
            iou_in_list.append(iou_in.cpu())
            iou_bothunique = metric_functions("iou")(data_both, label_both)
            iou_bothunique_list.append(iou_bothunique.cpu())

            ap_05_in = metric_functions("ap@0.5")(data_in, label_in)
            ap_05_in_list.append(ap_05_in)
            ap_05_bothunique = metric_functions("ap@0.5")(data_both, label_both)
            ap_05_bothunique_list.append(ap_05_bothunique)

            ap_07_in = metric_functions("ap@0.7")(data_in, label_in)
            ap_07_in_list.append(ap_07_in)
            ap_07_bothunique = metric_functions("ap@0.7")(data_both, label_both)
            ap_07_bothunique_list.append(ap_07_bothunique)

            eao_in = metric_functions("eao")(data_in, label_in)
            eao_in_list.append(eao_in.cpu())
            eao_bothunique = metric_functions("eao")(data_both, label_both)
            eao_bothunique_list.append(eao_bothunique.cpu())

            asd_in = asd_bothunique = assd_in = assd_bothunique = hd_in = hd_bothunique = hd95_in = hd95_bothunique = 0
            for j in range(data_in.shape[0]):
                asd_in += DistanceMetric(method_name="asd",
                                         voxelspacing=voxel,
                                         threshold=0.5)(data_in[j],
                                                        label_in[j])
                asd_bothunique += DistanceMetric(method_name="asd",
                                                 voxelspacing=voxel,
                                                 threshold=0.5)(data_both[j],
                                                                label_both[j])
                assd_in = DistanceMetric(method_name="assd",
                                         voxelspacing=voxel,
                                         threshold=0.5)(data_in[j],
                                                        label_in[j])
                assd_bothunique += DistanceMetric(method_name="assd",
                                                  voxelspacing=voxel,
                                                  threshold=0.5)(data_both[j],
                                                                 label_both[j])
                hd_in += DistanceMetric(method_name="hd",
                                        voxelspacing=voxel,
                                        threshold=0.5)(data_in[j], label_in[j])
                hd_bothunique += DistanceMetric(method_name="hd",
                                                voxelspacing=voxel,
                                                threshold=0.5)(data_both[j],
                                                               label_both[j])
                hd95_in += DistanceMetric(method_name="hd95",
                                          voxelspacing=voxel,
                                          threshold=0.5)(data_in[j],
                                                         label_in[j])
                hd95_bothunique += DistanceMetric(method_name="hd95",
                                                  voxelspacing=voxel,
                                                  threshold=0.5)(data_both[j],
                                                                 label_both[j])

            asd_in_list.append(asd_in / data_in.shape[0])
            asd_bothunique_list.append(asd_bothunique / data_in.shape[0])
            assd_in_list.append(assd_in / data_in.shape[0])
            assd_bothunique_list.append(assd_bothunique / data_in.shape[0])
            hd_in_list.append(hd_in / data_in.shape[0])
            hd_bothunique_list.append(hd_bothunique / data_in.shape[0])
            hd95_in_list.append(hd95_in / data_in.shape[0])
            hd95_bothunique_list.append(hd95_bothunique / data_in.shape[0])


    if True:
        test_loss_dict = dict({
            "dice_in":
            f'{np.array(dice_in_list).mean(axis=0):.2f}±{np.array(dice_in_list).std(axis=0):.2f}',
            "dice_bothunique":
            f'{np.array(dice_bothunique_list).mean(axis=0):.2f}±{np.array(dice_bothunique_list).std(axis=0):.2f}',
            "ac_in":
            f'{np.array(ac_in_list).mean(axis=0):.2f}±{np.array(ac_in_list).std(axis=0):.2f}',
            "ac_bothunique":
            f'{np.array(ac_bothunique_list).mean(axis=0):.2f}±{np.array(ac_bothunique_list).std(axis=0):.2f}',
            "pre_in":
            f'{np.array(pre_in_list).mean(axis=0):.2f}±{np.array(pre_in_list).std(axis=0):.2f}',
            "pre_bothunique":
            f'{np.array(pre_bothunique_list).mean(axis=0):.2f}±{np.array(pre_bothunique_list).std(axis=0):.2f}',
            "r_in":
            f'{np.array(r_in_list).mean(axis=0):.2f}±{np.array(r_in_list).std(axis=0):.2f}',
            "r_bothunique":
            f'{np.array(r_bothunique_list).mean(axis=0):.2f}±{np.array(r_bothunique_list).std(axis=0):.2f}',
            "f1_in":
            f'{np.array(f1_in_list).mean(axis=0):.2f}±{np.array(f1_in_list).std(axis=0):.2f}',
            "f1_bothunique":
            f'{np.array(f1_bothunique_list).mean(axis=0):.2f}±{np.array(f1_bothunique_list).std(axis=0):.2f}',
            "iou_in":
            f'{np.array(iou_in_list).mean(axis=0):.2f}±{np.array(iou_in_list).std(axis=0):.2f}',
            "iou_bothunique":
            f'{np.array(iou_bothunique_list).mean(axis=0):.2f}±{np.array(iou_bothunique_list).std(axis=0):.2f}',
            "ap@0.5_in":
            f'{np.array(ap_05_in_list).mean(axis=0):.2f}±{np.array(ap_05_in_list).std(axis=0):.2f}',
            "ap@0.5_bothunique":
            f'{np.array(ap_05_bothunique_list).mean(axis=0):.2f}±{np.array(ap_05_bothunique_list).std(axis=0):.2f}',
            "ap@0.7_in":
            f'{np.array(ap_07_in_list).mean(axis=0):.2f}±{np.array(ap_07_in_list).std(axis=0):.2f}',
            "ap@0.7_bothunique":
            f'{np.array(ap_07_bothunique_list).mean(axis=0):.2f}±{np.array(ap_07_bothunique_list).std(axis=0):.2f}',
            "eao_in":
            f'{np.array(eao_in_list).mean(axis=0):.2f}±{np.array(eao_in_list).std(axis=0):.2f}',
            "eao_bothunique":
            f'{np.array(eao_bothunique_list).mean(axis=0):.2f}±{np.array(eao_bothunique_list).std(axis=0):.2f}',
            "asd_in":
            f'{np.array(asd_in_list).mean(axis=0):.2f}±{np.array(asd_in_list).std(axis=0):.2f}',
            "asd_bothunique":
            f'{np.array(asd_bothunique_list).mean(axis=0):.2f}±{np.array(asd_bothunique_list).std(axis=0):.2f}',
            "assd_in":
            f'{np.array(assd_in_list).mean(axis=0):.2f}±{np.array(assd_in_list).std(axis=0):.2f}',
            "assd_bothunique":
            f'{np.array(assd_bothunique_list).mean(axis=0):.2f}±{np.array(assd_bothunique_list).std(axis=0):.2f}',
            "hd_in":
            f'{np.array(hd_in_list).mean(axis=0):.2f}±{np.array(hd_in_list).std(axis=0):.2f}',
            "hd_bothunique":
            f'{np.array(hd_bothunique_list).mean(axis=0):.2f}±{np.array(hd_bothunique_list).std(axis=0):.2f}',
            "hd95_in":
            f'{np.array(hd95_in_list).mean(axis=0):.2f}±{np.array(hd95_in_list).std(axis=0):.2f}',
            "hd95_bothunique":
            f'{np.array(hd95_bothunique_list).mean(axis=0):.2f}±{np.array(hd95_bothunique_list).std(axis=0):.2f}',
        })


        if True:
            values = np.stack([
            dice_in_list,
            dice_bothunique_list,
            ac_in_list,
            ac_bothunique_list,
            pre_in_list,
            pre_bothunique_list,
            r_in_list,
            r_bothunique_list,
            f1_in_list,
            f1_bothunique_list,
            iou_in_list,
            iou_bothunique_list,
            ap_05_in_list,
            ap_05_bothunique_list,
            ap_07_in_list,
            ap_07_bothunique_list,
            eao_in_list,
            eao_bothunique_list,
            asd_in_list,
            asd_bothunique_list,
            assd_in_list,
            assd_bothunique_list,
            hd_in_list,
            hd_bothunique_list,
            hd95_in_list,
            hd95_bothunique_list
            ])
            index = [
                "dice内膜",
                "dice外膜",
                "ac内膜",
                "ac外膜",
                "pre内膜",
                "pre外膜",
                "r内膜",
                "r外膜",
                "f1内膜",
                "f1外膜",
                "iou内膜",
                "iou外膜",
                "ap_05内膜",
                "ap_05外膜",
                "ap_07内膜",
                "ap_07外膜",
                "eao内膜",
                "eao外膜",
                "asd内膜",
                "asd外膜",
                "assd内膜",
                "assd外膜",
                "hd内膜",
                "hd外膜",
                "hd95内膜",
                "hd95外膜",
            ]
            columns = list(mask_data.keys())
            df = pd.DataFrame(values, index=index, columns=columns)
            with pd.ExcelWriter(f'{save_dir}/{model_name}_detail.xlsx') as writer:
                df.to_excel(writer,
                            sheet_name='Sheet1',
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)


        if False:
            df = pd.DataFrame(test_loss_dict, index=["均值±标准差"])
            with pd.ExcelWriter(f'{save_dir}/{model_name}.xlsx') as writer:
                df.to_excel(writer,
                            sheet_name='Sheet1',
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

        print(test_loss_dict)


model_name = "3dunet"
save_dir = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/count_index"

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values)

count_index(mask_3dunet_all, mask_all, spacing_pixels, model_name, save_dir)

{'dice_in': '0.91±0.07', 'dice_bothunique': '0.92±0.06', 'ac_in': '0.00±0.00', 'ac_bothunique': '0.00±0.00', 'pre_in': '0.89±0.10', 'pre_bothunique': '0.91±0.09', 'r_in': '0.94±0.04', 'r_bothunique': '0.94±0.03', 'f1_in': '0.91±0.07', 'f1_bothunique': '0.92±0.06', 'iou_in': '0.83±0.10', 'iou_bothunique': '0.86±0.09', 'ap@0.5_in': '0.97±0.09', 'ap@0.5_bothunique': '0.98±0.09', 'ap@0.7_in': '0.89±0.16', 'ap@0.7_bothunique': '0.94±0.14', 'eao_in': '0.91±0.04', 'eao_bothunique': '0.90±0.05', 'asd_in': '0.84±1.10', 'asd_bothunique': '0.79±1.04', 'assd_in': '0.01±0.01', 'assd_bothunique': '0.50±0.53', 'hd_in': '2.86±3.26', 'hd_bothunique': '2.82±3.16', 'hd95_in': '2.29±2.87', 'hd95_bothunique': '2.17±2.76'}


In [99]:
a = torch.tensor(mask_3dunet_all['_20210128075939_0818310.nii.gz']['in'])
b = torch.tensor(mask_rvm_all['_20210128075939_0818310.nii.gz']['in'])
c = cv2.resize(np.array(a).transpose((1, 2, 0)), (128, 128),
               interpolation=cv2.INTER_NEAREST).transpose((2, 0, 1))


In [108]:
a = [1, 2, 3, 4]
b = [3, 4, 5, 6]
values = np.stack([a, b])

In [114]:
pd.DataFrame(values, index=["nei", "wai"], columns=["11", "22", "33", "44"])

,11,22,33,44
nei,1,2,3,4
wai,3,4,5,6


In [115]:
index = ["内膜", "外膜"] * 3
index

['内膜', '外膜', '内膜', '外膜', '内膜', '外膜']